In [ ]:
from azureml.core import Workspace, Experiment, Dataset
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.train.estimator import Estimator
import json

In [ ]:
interactive_auth = InteractiveLoginAuthentication(tenant_id="39288a38-ff19-432c-8011-1cd9d0dff445")
ws = Workspace(subscription_id="793146d9-d4dc-4a73-9728-76c4ffd0cc0d", resource_group="rg_dynamics_test", workspace_name="resdynml1test", auth=interactive_auth)

In [ ]:
# load experiment cfg
with open("experiment_cfg.json", "r") as cfg_file:
    cfg = json.load(cfg_file)

## Score Script

In [ ]:
%%writefile ./src/score.py

import json
import numpy as np
import os
from azureml.core.model import Model
import joblib
from pipe import create_pipeline
import pandas as pd

def init():
    global model
    global regressor
    global pipelines
    model_path = Model.get_model_path('DummyModel')
    model = joblib.load(model_path)
    regressor_path = Model.get_model_path('DummyModelRegressor')
    regressor = joblib.load(regressor_path)
    pipeline_path = Model.get_model_path('DummyPipe')
    pipelines = joblib.load(pipeline_path)
    
def run(raw_data):
    
    # get input data
    data = json.loads(raw_data)
    
    # transform with pipeline
    X = pipelines['feature_pipe'].transform(pd.DataFrame(data))
    
    # make prediction
    y = model.predict(X)
    
    # predict duration
    y_dur = regressor.predict(X)
    
    response = [
        {
            'Products':
            [ 
                pipelines['target_pipe'].transformer_list[1][1].named_steps['target_encode'].col_cats[0][i] 
                for i in range(y.shape[1]) if y[j,i] == 1 
            ],
            'Duration':
                 y_dur[j,0]
        }        
            for j in range(y.shape[0])
    ]

    return response

## Deploy Webservice

In [ ]:
from azureml.core.webservice import AciWebservice, LocalWebservice
from azureml.core.model import InferenceConfig
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

cd = CondaDependencies.create()
cd.add_pip_package("azureml-defaults")
cd.add_pip_package('pyarrow==0.12.0')
cd.add_pip_package('joblib')
cd.add_pip_package('scikit-learn==0.20.3')
cd.save_to_file(base_directory='./score', conda_file_path='myenv.yml')

myenv = Environment.from_conda_specification(name="myenv", file_path="./score/myenv.yml")
inference_config = InferenceConfig(entry_script="score.py", environment=myenv, source_directory='./score')

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1,
                                               auth_enabled=False, # this flag generates API keys to secure access
                                               memory_gb=1,                                 
                                               description='Dummy Classifier for Product Prediction')

#aciconfig = LocalWebservice.deploy_configuration(port=1234)

service = Model.deploy(workspace=ws, 
                           name='dummy-product-prediction-aci',
                           models= [ws.models['DummyModel'], ws.models['DummyModelRegressor'], ws.models['DummyPipe']], 
                           inference_config=inference_config, 
                           deployment_config=aciconfig)

service.wait_for_deployment()
# print(service.state)